In [2]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

In [13]:
file = './data_results/4WT_all_fx.xlsx'
def recup_variable(file):
    a = pd.read_excel(file)
    data_variable = {}
    for var in a[(a["Type"]=='variable')&(a["Count"]>1)]['Name']:
        data_variable[var] = pd.read_excel(file,sheet_name=var,header=2)
    return data_variable
def recup_parameter(file):
    a = pd.read_excel(file)
    data_variable = {}
    for var in a[(a["Type"]=='parameter')&(a["Count"]>1)]['Name']:
        data_variable[var] = pd.read_excel(file,sheet_name=var,header=2)
    return data_variable
def recup_set(file):
    a = pd.read_excel(file)
    data_variable = {}
    for var in a[(a["Type"]=='set')&(a["Count"]>1)]['Name']:
        data_variable[var] = pd.read_excel(file,sheet_name=var,header=2)
    return data_variable
data_variable = recup_variable(file)
data_parameter = recup_parameter(file)
data_set = recup_set(file)

In [17]:
data_set['l'][data_set['l']['n.1']=='r1']['n']

2    j1
Name: n, dtype: object

In [12]:
data_variable['Qpipe'][data_variable['Qpipe']['n.1']=='r1']

,n,n.1,t,Lowerbound,Value,Upperbound,Marginal
48,j1,r1,t1,0,24.129543,inf,0.000000
49,j1,r1,t2,0,41.445874,inf,0.000000
50,j1,r1,t3,0,34.848880,inf,0.000000
51,j1,r1,t4,0,0.000000,inf,0.000000
52,j1,r1,t5,0,39.002441,inf,0.000000
53,j1,r1,t6,0,27.263068,inf,0.000000
54,j1,r1,t7,0,36.810990,inf,0.000000
55,j1,r1,t8,0,36.509205,inf,0.000000
56,j1,r1,t9,0,0.000000,inf,0.000911
57,j1,r1,t10,0,0.000000,inf,0.000911


In [28]:
reservoir = 'r1'
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x = data_parameter['demand'][data_parameter['demand']['r']==reservoir]['t'],
        y = -data_parameter['demand'][data_parameter['demand']['r']==reservoir]['Value'],
        stackgroup='one'
    )
)

fig.add_trace(
    go.Scatter(
        x = data_set['t']['dim1'],
        y = sum(data_variable['Qpipe'][(data_variable['Qpipe']['n.1']==reservoir)&(data_variable['Qpipe']['n']==n)]['Value'] for n in data_set['l'][data_set['l']['n.1']=='r1']['n']),
        stackgroup='two'
    )
)
fig.show()

In [51]:
import os

In [53]:
'out.xlsx'.endswith('.xlsx')

True

In [54]:
[file for file in os.listdir('./data_results') if file.endswith('.xlsx')]

['4WT_fx_night.xlsx', 'out.xlsx', '~$4WT_fx_night.xlsx']

In [31]:
data_parameter['psi'][data_parameter['psi']['degree']==2]['Value']

1   -0.006409
Name: Value, dtype: float64

In [50]:
data_parameter['psi']
X = np.linspace(0,100,100)
Y = data_parameter['psi'][data_parameter['psi']['degree']==2]['Value'].iloc[0]*X**2 + data_parameter['psi'][data_parameter['psi']['degree']==0]['Value'].iloc[0]
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x = X,
        y = Y,
        name = 'Courbe de gain de charge théorique'
    )
)
for c in data_variable['Qpompe']['c'].unique():
    for d in data_variable['Qpompe']['d'].unique():
        fig.add_trace(
            go.Scatter(
                x               =   data_variable['Qpompe'][(data_variable['Qpompe']['c']==c)&(data_variable['Qpompe']['d']==d)]['Value'],
                y               =   data_variable['Gpompe']['Value'],
                marker_color    =   color_map[d],
                name            =   f"Débit {c},{d}",
                # hoverinfo       =   data_variable['Qpompe'][(data_variable['Qpompe']['c']==c)&(data_variable['Qpompe']['d']==d)]['t'],
                mode='markers'
            )
        )
fig.update_layout(
    height= 800,
    hovermode='x'
)
fig.update_xaxes(range=[0,data_variable['Qpompe']['Value'].max()*1.1])
fig.update_yaxes(range=[data_variable['Gpompe']['Value'].min()*0.9,data_variable['Gpompe']['Value'].max()*1.1])
fig.show()

In [13]:
Z = 2
fig = go.Figure()
for r in data_variable['v']['n'].unique():
    fig.add_trace(
        go.Bar(
            x   =  data_variable['v'][data_variable['v']['n']==r]['t'],
            y   =  data_variable['v'][data_variable['v']['n']==r]['Value'],
            name=   r
        )
    )
for c in data_variable['Ppompe']['c'].unique():
    for d in data_variable['Ppompe']['d'].unique():
        fig.add_trace(
            go.Scatter(
                x   =   data_variable['Ppompe'][(data_variable['Ppompe']['c']==c)&(data_variable['Ppompe']['d']==d)]['t'],
                y   =   data_variable['Ppompe'][(data_variable['Ppompe']['c']==c)&(data_variable['Ppompe']['d']==d)]['Value'],
                yaxis="y2",
                name=   f"({c},{d})"
            )
        )
fig.update_layout(
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
            ),
        hovermode='x',
        yaxis=dict(
            title="m3" ,
            # range=[0,df["Consommation (MW)"].max()*1.1]
            ),
        yaxis2=dict(title="kW",
        # range=[df["Coût MWh"].min()*0.95,df["Coût MWh"].max()*1.1],
        anchor="free",
        overlaying="y",
        side="right",
        position=1
        ),
        title = f"Etat du réseau de distribution d'eau - Coût : {Z} €"
    )
fig.show()
# fig.write_html("result.html")

In [56]:
[file for file in os.listdir('./data_results') if file.endswith('.xlsx')][0]

'4WT_fx_night.xlsx'

In [64]:
data_variable['Charge'][data_variable['Charge']['n']=='s']['Value']

0     57.635219
1     54.299841
2     54.558968
3     54.949131
4     55.367206
5     55.807872
6     56.265675
7     56.385046
8     53.760826
9     60.929469
10    53.366659
11    58.832858
12    61.051633
13    60.923149
14    55.287573
15    60.908369
16    60.909363
17    63.073191
18    63.073191
19    63.073191
20    63.073191
21    58.391783
22    63.073191
23    59.646449
Name: Value, dtype: float64

In [23]:
color_map   =   {
    'p1'    :   'red',
    'p2'    :   'blue',
    'p3'    :   'green'
}
color_map_light   =   {
    'p1'    :   'lightcoral',
    'p2'    :   'lightblue',
    'p3'    :   'lightgreen'
}
fig = go.Figure()
for c in data_variable['Son']['c'].unique():
    for d in data_variable['Son']['d'].unique():
        fig.add_trace(
            go.Bar(
                x               =   data_variable['Son'][(data_variable['Son']['c']==c)&(data_variable['Son']['d']==d)]['t'],
                y               =   10*data_variable['Son'][(data_variable['Son']['c']==c)&(data_variable['Son']['d']==d)]['Value'],
                marker_color    =   color_map_light[d],
                name            =   f"Statut {c},{d}"
            )
        )
fig.update_layout(
    barmode="stack"
)
fig.add_trace(
    go.Scatter(
        x               =   data_variable['Gpompe']['t'],
        y               =   data_variable['Gpompe']['Value'],
        marker_color    =   'black',
        line_dash       =   'dash',
        name            =   f"Gain Charge {c},{d}"
    )
)
for c in data_variable['Qpompe']['c'].unique():
    for d in data_variable['Qpompe']['d'].unique():
        fig.add_trace(
            go.Scatter(
                x               =   data_variable['Qpompe'][(data_variable['Qpompe']['c']==c)&(data_variable['Qpompe']['d']==d)]['t'],
                y               =   data_variable['Qpompe'][(data_variable['Qpompe']['c']==c)&(data_variable['Qpompe']['d']==d)]['Value'],
                marker_color    =   color_map[d],
                name            =   f"Débit {c},{d}"
            )
        )
fig.update_layout(
    hovermode='x',
    title = f"Etat des pompes du réseau de distribution d'eau - Coût : {Z} € "
    )
fig.show()

In [ ]:
fig = go.Figure()
for n in data_variable['Charge']['n'].unique():
    fig.add_trace(
        go.Scatter(
            x = data_variable['Charge'][data_variable['Charge']['n']==n]['t'],
            y = data_variable['Charge'][data_variable['Charge']['n']==n]['Value'],
            name = n
        )
    )
fig.update_layout(
    legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
            ),
    hovermode='x',
    title = f"Charge dans le réseau de distribution d'eau - Coût : {Z} € ",
    yaxis_title = "Charge (m)"
)
fig.show()